# 🎵 deepPan Steel Pan Synthesizer

Interactive steel pan sound design in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/profLewis/deepPan/blob/main/deepPan_Colab.ipynb)

---

In [ ]:
# Install dependencies
!pip install -q numpy scipy ipywidgets

import numpy as np
from scipy.io import wavfile
from scipy import signal
import IPython.display as ipd
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Dropdown, Button, Output, HBox, VBox
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json

print("✓ Dependencies loaded!")

In [ ]:
# Note frequencies and mapping
SAMPLE_RATE = 44100

NOTE_FREQUENCIES = {
    'C': 261.63, 'C#': 277.18, 'Db': 277.18,
    'D': 293.66, 'D#': 311.13, 'Eb': 311.13,
    'E': 329.63, 'F': 349.23, 'F#': 369.99,
    'G': 392.00, 'G#': 415.30, 'Ab': 415.30,
    'A': 440.00, 'A#': 466.16, 'Bb': 466.16,
    'B': 493.88,
}

# All 29 notes on the tenor pan
TENOR_PAN_NOTES = [
    # Outer ring (octave 4)
    'F#4', 'B4', 'E4', 'A4', 'D4', 'G4', 'C4', 'F4', 'Bb4', 'Eb4', 'Ab4', 'C#4',
    # Central ring (octave 5)
    'F#5', 'B5', 'E5', 'A5', 'D5', 'G5', 'C5', 'F5', 'Bb5', 'Eb5', 'Ab5', 'C#5',
    # Inner ring (octave 6)
    'C#6', 'E6', 'D6', 'C6', 'Eb6'
]

def get_frequency(note_str):
    """Parse note string like 'C4' or 'F#5' and return frequency."""
    if len(note_str) >= 2:
        if len(note_str) >= 3 and note_str[1] in '#b':
            note_name = note_str[:2]
            octave = int(note_str[2:])
        else:
            note_name = note_str[0]
            octave = int(note_str[1:])
        base_freq = NOTE_FREQUENCIES.get(note_name, 440)
        return base_freq * (2 ** (octave - 4))
    return 440

print(f"✓ Tenor pan has {len(TENOR_PAN_NOTES)} notes")
print(f"  Outer ring (4): {', '.join(TENOR_PAN_NOTES[:12])}")
print(f"  Central ring (5): {', '.join(TENOR_PAN_NOTES[12:24])}")
print(f"  Inner ring (6): {', '.join(TENOR_PAN_NOTES[24:])}")

In [ ]:
# Default parameters
params = {
    'attack': 15,       # ms
    'decay': 500,       # ms
    'sustain': 20,      # %
    'release': 300,     # ms
    'fundamental': 100, # %
    'harmonic2': 30,    # %
    'harmonic3': 10,    # %
    'harmonic4': 5,     # %
    'sub_bass': 20,     # %
    'detune': 2,        # cents
    'filter': 6000,     # Hz
    'brightness': 50,   # %
    'duration': 1.5,    # seconds
    'volume': 85,       # %
}

# Presets
PRESETS = {
    'default': dict(attack=15, decay=500, sustain=20, release=300,
                    fundamental=100, harmonic2=30, harmonic3=10, harmonic4=5,
                    sub_bass=20, detune=2, filter=6000, brightness=50, duration=1.5, volume=85),
    'bright': dict(attack=5, decay=300, sustain=10, release=200,
                   fundamental=80, harmonic2=50, harmonic3=30, harmonic4=20,
                   sub_bass=10, detune=3, filter=10000, brightness=80, duration=1.5, volume=85),
    'mellow': dict(attack=30, decay=800, sustain=30, release=500,
                   fundamental=100, harmonic2=15, harmonic3=5, harmonic4=2,
                   sub_bass=30, detune=1, filter=3000, brightness=30, duration=1.5, volume=85),
    'bell': dict(attack=2, decay=1500, sustain=5, release=1000,
                 fundamental=70, harmonic2=60, harmonic3=40, harmonic4=30,
                 sub_bass=5, detune=5, filter=8000, brightness=60, duration=1.5, volume=85),
    'pluck': dict(attack=1, decay=200, sustain=0, release=100,
                  fundamental=100, harmonic2=40, harmonic3=20, harmonic4=10,
                  sub_bass=15, detune=0, filter=5000, brightness=50, duration=1.5, volume=85),
}

print("✓ Parameters initialized")
print(f"  Presets: {', '.join(PRESETS.keys())}")

In [ ]:
def generate_note(frequency, params):
    """Generate a steel pan note with given parameters."""
    duration = params['duration']
    t = np.linspace(0, duration, int(SAMPLE_RATE * duration), endpoint=False)
    
    # Convert parameters
    attack_time = params['attack'] / 1000
    decay_time = params['decay'] / 1000
    sustain_level = params['sustain'] / 100
    release_time = params['release'] / 1000
    detune_cents = params['detune']
    
    # Partial structure
    partials = [
        (0.50, params['sub_bass'] / 100, 1.0),
        (1.00 - detune_cents/1000, params['fundamental'] / 100 * 0.1, 1.0),
        (1.00, params['fundamental'] / 100, 1.0),
        (1.00 + detune_cents/1000, params['fundamental'] / 100 * 0.1, 1.0),
        (2.00, params['harmonic2'] / 100, 1.3),
        (3.00, params['harmonic3'] / 100, 1.8),
        (4.00, params['harmonic4'] / 100, 2.2),
    ]
    
    attack_samples = int(attack_time * SAMPLE_RATE)
    decay_samples = int(decay_time * SAMPLE_RATE)
    
    sound = np.zeros_like(t)
    
    for ratio, amp, decay_mult in partials:
        if amp < 0.01:
            continue
        
        freq = frequency * ratio
        if freq > SAMPLE_RATE / 2 - 500 or freq < 30:
            continue
        
        # ADSR envelope
        env = np.ones_like(t)
        
        if attack_samples > 0:
            attack = np.sin(np.linspace(0, np.pi/2, attack_samples)) ** 1.5
            env[:attack_samples] = attack
        
        decay_start = attack_samples
        decay_end = min(decay_start + decay_samples, len(t))
        if decay_end > decay_start:
            decay = np.linspace(1.0, sustain_level, decay_end - decay_start)
            env[decay_start:decay_end] = decay
        
        sustain_start = decay_end
        sustain_tau = decay_time / decay_mult
        if sustain_start < len(t):
            env[sustain_start:] = sustain_level * np.exp(
                -(t[sustain_start:] - t[sustain_start]) / sustain_tau
            )
        
        phase = np.random.uniform(0, 2 * np.pi)
        partial_sound = amp * env * np.sin(2 * np.pi * freq * t + phase)
        sound += partial_sound
    
    # Low-pass filter
    nyq = SAMPLE_RATE / 2
    cutoff = min(params['filter'], nyq - 100) / nyq
    if 0.01 < cutoff < 0.99:
        b, a = signal.butter(2, cutoff, btype='low')
        sound = signal.filtfilt(b, a, sound)
    
    # Brightness adjustment
    if params['brightness'] != 50:
        brightness_boost = (params['brightness'] - 50) / 50
        if brightness_boost > 0:
            high_cutoff = 2000 / nyq
            if high_cutoff < 0.99:
                b, a = signal.butter(1, high_cutoff, btype='high')
                highs = signal.filtfilt(b, a, sound)
                sound += highs * brightness_boost * 0.5
    
    # Soft limiting and normalize
    sound = np.tanh(sound * 1.2) / 1.2
    max_val = np.max(np.abs(sound))
    if max_val > 0:
        sound = sound / max_val * (params['volume'] / 100)
    
    return sound

print("✓ Synthesis engine ready")

In [ ]:
def play_note(note, params):
    """Generate and play a note."""
    freq = get_frequency(note)
    audio = generate_note(freq, params)
    return ipd.Audio(audio, rate=SAMPLE_RATE, autoplay=True)

def play_sequence(notes, params, bpm=120):
    """Generate a sequence of notes."""
    note_duration = 60 / bpm
    silence_samples = int(note_duration * SAMPLE_RATE)
    
    all_audio = []
    for note in notes.split():
        if note in (',,', '-', 'r', '.'):
            # Rest
            all_audio.append(np.zeros(silence_samples))
        else:
            freq = get_frequency(note)
            audio = generate_note(freq, params)
            # Trim or pad to beat length
            if len(audio) > silence_samples:
                audio = audio[:silence_samples]
            else:
                audio = np.pad(audio, (0, silence_samples - len(audio)))
            all_audio.append(audio)
    
    combined = np.concatenate(all_audio)
    return ipd.Audio(combined, rate=SAMPLE_RATE, autoplay=True)

print("✓ Playback functions ready")

---
## 🎹 Quick Play

Play individual notes or sequences:

In [ ]:
# Play a single note
play_note('C5', params)

In [ ]:
# Play a scale
play_sequence('C4 D4 E4 F4 G4 A4 B4 C5', params, bpm=120)

In [ ]:
# Play a melody
play_sequence('E5 E5 F5 G5 G5 F5 E5 D5 C5 C5 D5 E5 E5 ,, D5 D5', params, bpm=140)

---
## 🎛️ Interactive Synthesizer

Adjust parameters with sliders:

In [ ]:
# Create interactive synthesizer
output = Output()

# Sliders
note_dropdown = Dropdown(options=TENOR_PAN_NOTES, value='C5', description='Note:')
preset_dropdown = Dropdown(options=list(PRESETS.keys()), value='default', description='Preset:')

attack_slider = IntSlider(min=1, max=200, value=params['attack'], description='Attack (ms)')
decay_slider = IntSlider(min=50, max=2000, value=params['decay'], description='Decay (ms)')
sustain_slider = IntSlider(min=0, max=100, value=params['sustain'], description='Sustain (%)')
release_slider = IntSlider(min=50, max=2000, value=params['release'], description='Release (ms)')

fundamental_slider = IntSlider(min=0, max=100, value=params['fundamental'], description='Fundamental')
harmonic2_slider = IntSlider(min=0, max=100, value=params['harmonic2'], description='Harmonic 2')
harmonic3_slider = IntSlider(min=0, max=100, value=params['harmonic3'], description='Harmonic 3')
harmonic4_slider = IntSlider(min=0, max=100, value=params['harmonic4'], description='Harmonic 4')
sub_bass_slider = IntSlider(min=0, max=100, value=params['sub_bass'], description='Sub Bass')

detune_slider = IntSlider(min=0, max=20, value=params['detune'], description='Detune (ct)')
filter_slider = IntSlider(min=500, max=10000, value=params['filter'], description='Filter (Hz)')
brightness_slider = IntSlider(min=0, max=100, value=params['brightness'], description='Brightness')
volume_slider = IntSlider(min=0, max=100, value=params['volume'], description='Volume (%)')

def update_params():
    params['attack'] = attack_slider.value
    params['decay'] = decay_slider.value
    params['sustain'] = sustain_slider.value
    params['release'] = release_slider.value
    params['fundamental'] = fundamental_slider.value
    params['harmonic2'] = harmonic2_slider.value
    params['harmonic3'] = harmonic3_slider.value
    params['harmonic4'] = harmonic4_slider.value
    params['sub_bass'] = sub_bass_slider.value
    params['detune'] = detune_slider.value
    params['filter'] = filter_slider.value
    params['brightness'] = brightness_slider.value
    params['volume'] = volume_slider.value

def load_preset(change):
    preset = PRESETS[preset_dropdown.value]
    attack_slider.value = preset['attack']
    decay_slider.value = preset['decay']
    sustain_slider.value = preset['sustain']
    release_slider.value = preset['release']
    fundamental_slider.value = preset['fundamental']
    harmonic2_slider.value = preset['harmonic2']
    harmonic3_slider.value = preset['harmonic3']
    harmonic4_slider.value = preset['harmonic4']
    sub_bass_slider.value = preset['sub_bass']
    detune_slider.value = preset['detune']
    filter_slider.value = preset['filter']
    brightness_slider.value = preset['brightness']
    volume_slider.value = preset['volume']

preset_dropdown.observe(load_preset, names='value')

def play_current(b):
    update_params()
    with output:
        clear_output(wait=True)
        display(play_note(note_dropdown.value, params))

play_btn = Button(description='Play Note', button_style='success')
play_btn.on_click(play_current)

# Layout - use widgets.HTML for ipywidgets containers
controls = VBox([
    widgets.HTML('<h3>Note Selection</h3>'),
    HBox([note_dropdown, preset_dropdown, play_btn]),
    widgets.HTML('<h3>Envelope (ADSR)</h3>'),
    HBox([attack_slider, decay_slider]),
    HBox([sustain_slider, release_slider]),
    widgets.HTML('<h3>Harmonics</h3>'),
    HBox([fundamental_slider, harmonic2_slider]),
    HBox([harmonic3_slider, harmonic4_slider]),
    sub_bass_slider,
    widgets.HTML('<h3>Character</h3>'),
    HBox([detune_slider, filter_slider]),
    HBox([brightness_slider, volume_slider]),
    widgets.HTML('<h3>Output</h3>'),
    output
])

display(controls)

---
## 💾 Save / Load Parameters

In [ ]:
# Save current parameters to JSON
def save_params(filename='synth_params.json'):
    update_params()
    with open(filename, 'w') as f:
        json.dump(params, f, indent=2)
    print(f"✓ Saved parameters to {filename}")
    print(json.dumps(params, indent=2))

# Load parameters from JSON
def load_params(filename='synth_params.json'):
    global params
    with open(filename, 'r') as f:
        loaded = json.load(f)
    params.update(loaded)
    load_preset(None)  # Update sliders
    print(f"✓ Loaded parameters from {filename}")

# Example: save current settings
save_params('my_sound.json')

In [ ]:
# Generate CLI command for generate_sounds.py
def get_cli_command():
    update_params()
    defaults = PRESETS['default']
    cmd = 'python generate_sounds.py'
    
    for key, value in params.items():
        if value != defaults.get(key):
            cli_key = key.replace('_', '-')
            cmd += f' --{cli_key} {value}'
    
    return cmd

print("Equivalent CLI command:")
print(get_cli_command())

---
## 🎶 Sequence Player

In [ ]:
# Interactive sequence player
seq_output = Output()

sequence_input = widgets.Text(
    value='C4 E4 G4 C5 G4 E4 C4',
    description='Sequence:',
    layout=widgets.Layout(width='80%')
)

bpm_slider = IntSlider(min=60, max=200, value=120, description='BPM:')

def play_seq(b):
    update_params()
    with seq_output:
        clear_output(wait=True)
        display(play_sequence(sequence_input.value, params, bpm_slider.value))

play_seq_btn = Button(description='Play Sequence', button_style='success')
play_seq_btn.on_click(play_seq)

display(VBox([
    sequence_input,
    HBox([bpm_slider, play_seq_btn]),
    widgets.HTML('<p style="color: gray; font-size: 12px;">Notes: C4-B4 (outer), C5-B5 (central), C6-Eb6 (inner) | Rest: ,,</p>'),
    seq_output
]))

---
## Audio Analysis

Upload a WAV file to analyze and extract synthesis parameters:

In [ ]:
from scipy.fft import fft, fftfreq

NOTE_NAMES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def freq_to_note(freq):
    """Convert frequency to note name and octave."""
    if freq <= 0:
        return None, None
    midi_note = int(round(69 + 12 * np.log2(freq / 440.0)))
    octave = (midi_note // 12) - 1
    note_idx = midi_note % 12
    return NOTE_NAMES[note_idx], octave

def detect_pitch(audio, sample_rate, min_freq=50, max_freq=2000):
    """Detect fundamental frequency using autocorrelation and FFT."""
    # Use a segment from the sustain portion
    seg_start = int(0.05 * sample_rate)
    seg_length = int(0.2 * sample_rate)
    seg_end = min(seg_start + seg_length, len(audio))
    
    if seg_end <= seg_start:
        segment = audio
    else:
        segment = audio[seg_start:seg_end].copy()
    
    # Normalize
    segment = segment - np.mean(segment)
    if np.max(np.abs(segment)) > 0:
        segment = segment / np.max(np.abs(segment))
    
    # Autocorrelation method
    min_lag = int(sample_rate / max_freq)
    max_lag = min(int(sample_rate / min_freq), len(segment) - 1)
    
    corr = np.correlate(segment, segment, mode='full')
    corr = corr[len(corr)//2:]
    
    if max_lag > len(corr):
        max_lag = len(corr) - 1
    
    corr_segment = corr[min_lag:max_lag]
    if len(corr_segment) == 0:
        return 0
    
    peak_idx = np.argmax(corr_segment) + min_lag
    freq_autocorr = sample_rate / peak_idx if peak_idx > 0 else 0
    
    # FFT method with harmonic product spectrum
    n = len(segment)
    window = np.hanning(n)
    spectrum = np.abs(fft(segment * window))[:n//2]
    freqs = fftfreq(n, 1/sample_rate)[:n//2]
    
    # Harmonic product spectrum
    hps = spectrum.copy()
    for h in range(2, 5):
        decimated = spectrum[::h]
        hps[:len(decimated)] *= decimated
    
    freq_mask = (freqs >= min_freq) & (freqs <= max_freq)
    valid_hps = hps.copy()
    valid_hps[~freq_mask] = 0
    
    peak_idx = np.argmax(valid_hps)
    freq_fft = freqs[peak_idx] if peak_idx > 0 else 0
    
    # Use FFT if autocorrelation seems off
    if abs(freq_fft - freq_autocorr) > freq_autocorr * 0.1:
        return freq_fft
    return (freq_autocorr + freq_fft) / 2

def analyze_harmonics(audio, sample_rate, fundamental_freq):
    """Analyze harmonic content relative to fundamental."""
    seg_start = int(0.1 * sample_rate)
    seg_length = int(0.3 * sample_rate)
    seg_end = min(seg_start + seg_length, len(audio))
    
    segment = audio[seg_start:seg_end] if seg_end > seg_start else audio
    n = len(segment)
    window = np.hanning(n)
    spectrum = np.abs(fft(segment * window))[:n//2]
    freqs = fftfreq(n, 1/sample_rate)[:n//2]
    
    harmonics = {}
    harmonic_ratios = [0.5, 1.0, 2.0, 3.0, 4.0]
    harmonic_names = ['sub_bass', 'fundamental', 'harmonic2', 'harmonic3', 'harmonic4']
    
    fund_amp = 0
    for ratio, name in zip(harmonic_ratios, harmonic_names):
        target_freq = fundamental_freq * ratio
        tolerance = fundamental_freq * 0.05
        freq_mask = (freqs >= target_freq - tolerance) & (freqs <= target_freq + tolerance)
        
        if np.any(freq_mask):
            amp = np.max(spectrum[freq_mask])
            harmonics[name] = amp
            if name == 'fundamental':
                fund_amp = amp
        else:
            harmonics[name] = 0
    
    # Normalize to fundamental
    if fund_amp > 0:
        for name in harmonic_names:
            harmonics[name] = min(100, int((harmonics[name] / fund_amp) * 100))
    
    return harmonics

def analyze_envelope(audio, sample_rate):
    """Estimate ADSR envelope parameters."""
    envelope = np.abs(audio)
    window_size = int(0.01 * sample_rate)
    if window_size > 1:
        kernel = np.ones(window_size) / window_size
        envelope = np.convolve(envelope, kernel, mode='same')
    
    max_amp = np.max(envelope)
    if max_amp > 0:
        envelope = envelope / max_amp
    
    # Attack
    peak_idx = np.argmax(envelope)
    attack_idx = 0
    for i in range(peak_idx):
        if envelope[i] >= 0.9:
            attack_idx = i
            break
    attack_ms = (attack_idx / sample_rate) * 1000
    
    # Sustain level
    sustain_start = int(0.3 * len(envelope))
    sustain_end = int(0.6 * len(envelope))
    sustain_level = np.mean(envelope[sustain_start:sustain_end]) if sustain_end > sustain_start else 0.2
    
    # Decay
    decay_idx = peak_idx
    for i in range(peak_idx, len(envelope)):
        if envelope[i] <= sustain_level * 1.1:
            decay_idx = i
            break
    decay_ms = ((decay_idx - peak_idx) / sample_rate) * 1000
    
    # Release
    release_start = int(0.7 * len(envelope))
    if release_start < len(envelope):
        release_segment = envelope[release_start:]
        threshold = release_segment[0] * 0.1 if len(release_segment) > 0 else 0
        release_idx = len(release_segment)
        for i, val in enumerate(release_segment):
            if val <= threshold:
                release_idx = i
                break
        release_ms = (release_idx / sample_rate) * 1000
    else:
        release_ms = 300
    
    return {
        'attack': max(1, min(200, int(attack_ms))),
        'decay': max(50, min(2000, int(decay_ms))),
        'sustain': max(0, min(100, int(sustain_level * 100))),
        'release': max(50, min(2000, int(release_ms)))
    }

def estimate_filter_brightness(audio, sample_rate, fundamental_freq):
    """Estimate filter cutoff and brightness."""
    n = len(audio)
    window = np.hanning(n)
    spectrum = np.abs(fft(audio * window))[:n//2]
    freqs = fftfreq(n, 1/sample_rate)[:n//2]
    
    # Spectral centroid
    if np.sum(spectrum) > 0:
        centroid = np.sum(freqs * spectrum) / np.sum(spectrum)
    else:
        centroid = 1000
    
    # Rolloff frequency (85% energy)
    cumsum = np.cumsum(spectrum)
    total = cumsum[-1]
    if total > 0:
        rolloff_idx = np.where(cumsum >= 0.85 * total)[0]
        rolloff_freq = freqs[rolloff_idx[0]] if len(rolloff_idx) > 0 else 6000
    else:
        rolloff_freq = 6000
    
    filter_cutoff = max(500, min(10000, int(rolloff_freq)))
    brightness_ratio = centroid / fundamental_freq if fundamental_freq > 0 else 2
    brightness = max(0, min(100, int((brightness_ratio - 1) * 25 + 50)))
    
    return {'filter': filter_cutoff, 'brightness': brightness}

def analyze_audio_file(filepath):
    """Analyze an audio file and extract synthesis parameters."""
    sample_rate, audio = wavfile.read(filepath)
    
    # Convert to mono float
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    if audio.dtype == np.int16:
        audio = audio.astype(np.float32) / 32768.0
    elif audio.dtype == np.int32:
        audio = audio.astype(np.float32) / 2147483648.0
    
    # Detect pitch
    fundamental_freq = detect_pitch(audio, sample_rate)
    note_name, octave = freq_to_note(fundamental_freq)
    detected_note = f"{note_name}{octave}" if note_name else "Unknown"
    
    # Check if on tenor pan
    on_pan = detected_note in TENOR_PAN_NOTES
    
    # Analyze components
    harmonics = analyze_harmonics(audio, sample_rate, fundamental_freq)
    envelope = analyze_envelope(audio, sample_rate)
    filter_bright = estimate_filter_brightness(audio, sample_rate, fundamental_freq)
    
    # Build params
    extracted_params = {
        'attack': envelope['attack'],
        'decay': envelope['decay'],
        'sustain': envelope['sustain'],
        'release': envelope['release'],
        'fundamental': harmonics.get('fundamental', 100),
        'harmonic2': harmonics.get('harmonic2', 30),
        'harmonic3': harmonics.get('harmonic3', 10),
        'harmonic4': harmonics.get('harmonic4', 5),
        'sub_bass': harmonics.get('sub_bass', 20),
        'detune': 2,
        'filter': filter_bright['filter'],
        'brightness': filter_bright['brightness'],
        'duration': len(audio) / sample_rate,
        'volume': 85
    }
    
    return {
        'detected_note': detected_note,
        'frequency': round(fundamental_freq, 2),
        'on_pan': on_pan,
        'params': extracted_params,
        'audio': audio,
        'sample_rate': sample_rate
    }

print("Audio analysis functions ready")

In [ ]:
# Interactive audio analysis with file upload (supports multiple files)
from google.colab import files
import matplotlib.pyplot as plt

analysis_output = Output()
all_results = []  # Store multiple analysis results
selected_idx = 0

def upload_and_analyze(b):
    global all_results, selected_idx, params
    with analysis_output:
        clear_output(wait=True)
        print("Upload WAV file(s)... (you can select multiple)")
        uploaded = files.upload()
        
        if not uploaded:
            print("No files uploaded")
            return
        
        all_results = []
        for filename in uploaded.keys():
            print(f"\nAnalyzing {filename}...")
            try:
                result = analyze_audio_file(filename)
                result['filename'] = filename
                all_results.append(result)
            except Exception as e:
                print(f"  Error: {e}")
        
        if not all_results:
            print("No files could be analyzed")
            return
        
        # Print summary table
        print(f"\n{'='*70}")
        print(f"ANALYSIS SUMMARY ({len(all_results)} files)")
        print(f"{'='*70}")
        print(f"{'#':<3} {'File':<25} {'Note':<6} {'Freq':>8} {'On Pan':<6}")
        print("-"*70)
        
        for i, r in enumerate(all_results):
            fname = r['filename'][:23]
            on_pan = "Yes" if r['on_pan'] else "No"
            print(f"{i:<3} {fname:<25} {r['detected_note']:<6} {r['frequency']:>8.1f} {on_pan:<6}")
        
        print("-"*70)
        on_pan_count = sum(1 for r in all_results if r['on_pan'])
        print(f"Total: {len(all_results)} files, {on_pan_count} notes on tenor pan")
        
        # Show first result details
        selected_idx = 0
        show_result_details(0)

def show_result_details(idx):
    global selected_idx
    selected_idx = idx
    result = all_results[idx]
    
    print(f"\n{'='*50}")
    print(f"DETAILS: {result['filename']}")
    print(f"{'='*50}")
    print(f"Detected Note: {result['detected_note']}")
    print(f"Frequency: {result['frequency']} Hz")
    print(f"On Tenor Pan: {'Yes' if result['on_pan'] else 'No'}")
    
    p = result['params']
    print(f"\n--- Envelope (ADSR) ---")
    print(f"Attack:  {p['attack']} ms")
    print(f"Decay:   {p['decay']} ms")
    print(f"Sustain: {p['sustain']}%")
    print(f"Release: {p['release']} ms")
    
    print(f"\n--- Harmonics ---")
    print(f"Fundamental: {p['fundamental']}%")
    print(f"Harmonic 2:  {p['harmonic2']}%")
    print(f"Harmonic 3:  {p['harmonic3']}%")
    print(f"Harmonic 4:  {p['harmonic4']}%")
    print(f"Sub Bass:    {p['sub_bass']}%")
    
    print(f"\n--- Character ---")
    print(f"Filter:     {p['filter']} Hz")
    print(f"Brightness: {p['brightness']}%")
    
    # Plot waveform and spectrum
    fig, axes = plt.subplots(2, 1, figsize=(10, 4))
    
    t = np.arange(len(result['audio'])) / result['sample_rate']
    axes[0].plot(t, result['audio'], color='orange', linewidth=0.5)
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f'Waveform: {result["detected_note"]} ({result["frequency"]} Hz)')
    axes[0].grid(True, alpha=0.3)
    
    n = min(8192, len(result['audio']))
    spectrum = np.abs(fft(result['audio'][:n] * np.hanning(n)))[:n//2]
    freqs = fftfreq(n, 1/result['sample_rate'])[:n//2]
    axes[1].plot(freqs[:2000], spectrum[:2000], color='cyan', linewidth=0.5)
    axes[1].axvline(result['frequency'], color='red', linestyle='--', label=f'F0={result["frequency"]}Hz')
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Magnitude')
    axes[1].set_title('Spectrum')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def apply_analysis(b):
    global params
    if not all_results:
        print("No analysis result to apply")
        return
    
    result = all_results[selected_idx]
    for key, value in result['params'].items():
        if key in params:
            params[key] = value
    
    load_preset(None)
    print(f"Parameters from {result['filename']} applied to synthesizer!")
    print(f"Try playing {result['detected_note']} to compare")

def play_original(b):
    if not all_results:
        print("No audio loaded")
        return
    result = all_results[selected_idx]
    with analysis_output:
        display(ipd.Audio(result['audio'], rate=result['sample_rate'], autoplay=True))

# File selector dropdown (for multiple files)
file_selector = Dropdown(options=[], description='File:')

def on_file_select(change):
    if all_results and change['new'] is not None:
        with analysis_output:
            show_result_details(change['new'])

file_selector.observe(on_file_select, names='value')

upload_btn = Button(description='Upload & Analyze WAV(s)', button_style='warning')
upload_btn.on_click(upload_and_analyze)

apply_btn = Button(description='Apply to Synth', button_style='success')
apply_btn.on_click(apply_analysis)

play_orig_btn = Button(description='Play Original', button_style='info')
play_orig_btn.on_click(play_original)

display(VBox([
    HBox([upload_btn, apply_btn, play_orig_btn]),
    analysis_output
]))

In [ ]:
# Download and analyze example steel pan sample from the web
import urllib.request

def download_sample(url, filename='sample.wav'):
    """Download a WAV sample from URL."""
    print(f"Downloading {url}...")
    urllib.request.urlretrieve(url, filename)
    print(f"Saved to {filename}")
    return filename

# Example: Download a steel drum sample from Free Wave Samples
# (You can replace this URL with any WAV file URL)
sample_url = "https://freewavesamples.com/files/Steel-Drum-C4.wav"

try:
    local_file = download_sample(sample_url, 'steel_drum_sample.wav')
    result = analyze_audio_file(local_file)
    
    print(f"\nDetected: {result['detected_note']} ({result['frequency']} Hz)")
    print(f"On pan: {'Yes' if result['on_pan'] else 'No'}")
    print(f"\nExtracted parameters:")
    print(json.dumps(result['params'], indent=2))
    
    # Play the sample
    display(ipd.Audio(result['audio'], rate=result['sample_rate']))
except Exception as e:
    print(f"Could not download sample: {e}")
    print("You can upload your own WAV file using the button above")

In [ ]:
from google.colab import files

def download_note(note, filename=None):
    """Generate a note and download as WAV file."""
    update_params()
    freq = get_frequency(note)
    audio = generate_note(freq, params)
    
    # Convert to 16-bit stereo
    audio_int = (audio * 32767).astype(np.int16)
    stereo = np.column_stack((audio_int, audio_int))
    
    if filename is None:
        filename = f"{note.replace('#', 's')}.wav"
    
    wavfile.write(filename, SAMPLE_RATE, stereo)
    files.download(filename)
    print(f"✓ Downloaded {filename}")

# Download a single note
download_note('C5')

In [ ]:
def download_all_notes():
    """Generate and download all 29 tenor pan notes."""
    import zipfile
    import os
    
    update_params()
    
    # Create temp directory
    os.makedirs('sounds', exist_ok=True)
    
    for note in TENOR_PAN_NOTES:
        freq = get_frequency(note)
        audio = generate_note(freq, params)
        audio_int = (audio * 32767).astype(np.int16)
        stereo = np.column_stack((audio_int, audio_int))
        
        filename = f"sounds/{note.replace('#', 's')}.wav"
        wavfile.write(filename, SAMPLE_RATE, stereo)
        print(f"  Generated {note}")
    
    # Save params
    with open('sounds/params.json', 'w') as f:
        json.dump(params, f, indent=2)
    
    # Create zip
    with zipfile.ZipFile('deepPan_sounds.zip', 'w') as zf:
        for f in os.listdir('sounds'):
            zf.write(f'sounds/{f}', f)
    
    files.download('deepPan_sounds.zip')
    print(f"\n✓ Downloaded deepPan_sounds.zip with {len(TENOR_PAN_NOTES)} notes")

# Uncomment to generate all notes:
# download_all_notes()

---

## Links

- [GitHub Repository](https://github.com/profLewis/deepPan)
- [Browser Synthesizer](https://proflewis.github.io/deepPan/synth.html) (if GitHub Pages enabled)
- [Interactive Player](https://proflewis.github.io/deepPan/index.html)